# hashed-ezkl

Here's an example leveraging EZKL whereby the inputs to the model, and the model params themselves, are hashed inside a circuit.

In this setup:
- the hashes are publicly known to the prover and verifier
- the hashes serve as "public inputs" (a.k.a instances) to the circuit

We leave the outputs of the model as public as well (known to the  verifier and prover). 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [1]:
from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


Now we define our model. It is a humble model with but a conv layer and a $ReLU$ non-linearity, but it is a model nonetheless

In [2]:
import torch
# Defines the model
# we got convs, we got relu, 
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)

        return x


circuit = MyModel()

# this is where you'd train your model




We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input file.

You can replace the random `x` with real data if you so wish. 

In [3]:
x = 0.1*torch.rand(1,*[3, 8, 8], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open("input.json", 'w' ))



================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

There are currently 4 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this). 
- `private`: known only to the prover
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash. 
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.

Here we create the following setup:
- `input_visibility`: "hashed"
- `param_visibility`: "hashed"
- `output_visibility`: public

We encourage you to play around with other setups :) 

Shoutouts: 

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip. 
- [timeofey](https://github.com/timoftime) for providing inspiration in our developement of the el-gamal encryption circuit in Halo2. 

In [4]:
import ezkl

model_path = os.path.join('network.onnx')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "hashed"
run_args.param_visibility = "hashed"
run_args.output_visibility = "public"





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [5]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

DEBUG tract_onnx.model 2023-07-21 17:07:19,661 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:19,665 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:19,666 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:19,667 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:19,668 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:19,668 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:19,669 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:19,669 change_axes.rs:76   Considering change Axis

In [18]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [(0.1*torch.rand(10, *[3, 8, 8])).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

DEBUG tract_onnx.model 2023-07-21 17:09:41,452 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:41,453 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:41,454 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:41,455 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:41,456 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:41,457 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:41,457 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:41,458 change_axes.rs:76   Considering change Axis

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [7]:
res = ezkl.get_srs(srs_path, settings_path)


DEBUG reqwest.connect 2023-07-21 17:07:42,045 connect.rs:429 starting new connection: https://trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com/
INFO ezkl.execute 2023-07-21 17:07:45,158 execute.rs:502 SRS downloaded


We now need to generate the (partial) circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [16]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = ezkl.gen_witness(data_path, model_path, witness_path, settings_path = settings_path)

DEBUG tract_onnx.model 2023-07-21 17:09:24,770 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:24,772 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:24,772 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:24,773 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:24,773 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:24,774 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:24,774 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:24,775 change_axes.rs:76   Considering change Axis

As a sanity check you can "mock prove" (i.e check that all the constraints of the circuit match without generate a full proof). 

In [17]:


res = ezkl.mock(witness_path, model_path, settings_path)

DEBUG tract_onnx.model 2023-07-21 17:09:27,436 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:27,438 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:09:27,439 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:27,439 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:27,440 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:27,440 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:27,440 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:09:27,441 change_axes.rs:76   Considering change Axis

Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [10]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

DEBUG tract_onnx.model 2023-07-21 17:07:46,247 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:46,249 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:46,250 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:46,250 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:46,251 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:46,251 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:46,251 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:46,251 change_axes.rs:76   Considering change Axis

Now we generate a full proof. 

In [11]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

DEBUG tract_onnx.model 2023-07-21 17:07:49,827 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:49,829 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-21 17:07:49,837 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:49,847 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:49,848 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:49,848 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:49,849 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-21 17:07:49,850 change_axes.rs:76   Considering change Axis

{'instances': [[[16, 0, 0, 0]], [[4172702715194216444, 4343298861975152182, 16572759352690916491, 998843874258490695], [0, 0, 0, 0]]], 'proof': '1cee08d4ecae0ee3760f59955cc816977bd70b35887cf1fcfd00f89c11dc44bd0377d9dfe8a91c4f0fdefb9ca042c2b7c04d0d0aa078e77d311ddb2651b4c70e1f19036bcce17e99441708bfc632486534fef9587dc613c111911ace9fcd00db279fbd7f9098d57fc95798debcf76c0a3043ce57d191481edd6c6c0618ea22f00c52589d988f1fc061b839ddf8c21e14ab5f7d3350beb4887954920663a941200b6daff88327d209d35e28e33133316004687f46a708626c1215fd1124bbdd302cf6bcb59ee14bac2c2d8a8e3129f67adfebeee574d6ee3b47c0a0241b754ff703897c28439d116f6901b76cbe5bd5dda3237ca26f13546dbaaf9ab9dfad127b02fce1add7daed5a4f434ed8b92c33688d5578e23d0cb1fec15136c5ba486713086dee885b87fad30fba965a5bd28c6b7519d201a9babcdf64e505e4cd007347077e2b098f9d29776a36654bc4d3442ee9a64e3847e27e11e817aaa9341e806c164f1df9d9d50f5182656dd4e56502538c2daf846973a70c265d2074faaa9e85112f825e378dbe95262b1d4514b5d872fc3583389158a3661d71f49ce3d8810c2fb2cf7c295d81938b2e249

And verify it as a sanity check. 

In [12]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

INFO ezkl.pfsys.srs 2023-07-21 17:07:53,983 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:07:53,987 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-21 17:07:53,988 mod.rs:498 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-21 17:07:53,989 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-21 17:07:53,990 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.ops.chip 2023-07-21 17:07:53,990 chip.rs:345 assigning lookup output
INFO ezkl.execute 2023-07-21 17:07:54,002 execute.rs:1593 verify took 0.5
INFO ezkl.execute 2023-07-21 17:07:54,003 execute.rs:1598 verified: true


verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [13]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True


INFO ezkl.execute 2023-07-21 17:07:54,009 execute.rs:80 checking solc installation..
DEBUG ezkl.execute 2023-07-21 17:07:54,044 execute.rs:84 solc output: Output {
    status: ExitStatus(
        unix_wait_status(
            0,
        ),
    ),
    stdout: "solc, the solidity compiler commandline interface\nVersion: 0.8.20+commit.a1b79de6.Darwin.appleclang\n",
    stderr: "",
}
DEBUG ezkl.execute 2023-07-21 17:07:54,045 execute.rs:86 solc output success: true
DEBUG ezkl.execute 2023-07-21 17:07:54,045 execute.rs:93 solc check passed, proceeding
INFO ezkl.pfsys.srs 2023-07-21 17:07:54,046 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:07:54,051 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-21 17:07:54,053 mod.rs:498 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-21 17:07:54,053 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-21 17:07:54,053 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.op

## Verify on the evm

In [14]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

INFO ezkl.execute 2023-07-21 17:07:54,406 execute.rs:80 checking solc installation..
DEBUG reqwest.connect 2023-07-21 17:07:54,461 connect.rs:429 starting new connection: http://127.0.0.1:3030/


RuntimeError: Failed to run deploy_evm: error sending request for url (http://127.0.0.1:3030/): error trying to connect: tcp connect error: Connection refused (os error 61)

In [ ]:
# make sure anvil is running locally
# $ anvil -p 3030

res = ezkl.verify_evm(
    proof_path,
    addr,
    "http://127.0.0.1:3030"
)
assert res == True